In [ ]:
import tweepy

try:
    import json
except ImportError:
    import simplejson as json

import pymongo

In [ ]:
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
tweets = api.search(q=['coronavirus'], count=100, tweet_mode="extended")
'''
for tweet in tweets:
    print(json.dumps(tweet._json, sort_keys=True, indent=3, ensure_ascii=False))
'''
print(json.dumps(tweets[0]._json, indent=3, ensure_ascii=False))

In [ ]:
tweets_data = [tweet._json for tweet in tweets]

# Iniciando a conexão com o MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['coronaVirus']
collection = db.tweets

In [ ]:
collection.insert_many(tweets_data)

In [ ]:
number_of_tweets = tweets.count

In [ ]:
tweets_collection = tweets.copy()

In [ ]:
print(
    f"https://twitter.com/{tweets[0]._json['user']['screen_name']}/status/{tweets[0]._json['id']}")

In [ ]:
tweets[0]._json['full_text']

In [ ]:
while(len(tweets) != 0):
    try:
        tweets = api.search(q=['coronavirus'],
                            count=100,
                            tweet_mode="extended",
                            max_id=tweets[len(tweets)-1]._json['id']-1)

        tweets_data = [tweet._json for tweet in tweets]
        collection.insert_many(tweets_data)

        tweets_collection = tweets_collection + tweets
        number_of_tweets = number_of_tweets + tweets.count
        print("EXTRAINDO " + str(number_of_tweets) + " tweets")

    except tweepy.RateLimitError:
        print("\nEXTRACAO CONCLUIDA\n")
        break

In [ ]:
tweets_collection[-1]._json['full_text']